In [8]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from skimage.feature import canny
from sklearn.preprocessing import LabelEncoder

In [2]:
# EDGE BASED SEGMENTATION
def preprocess1(image):
  # resizing
  image = cv2.resize(image,(96,96))
  # gray scaling
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  return image

In [3]:
rootdir = 'archive/Sign Language for Numbers'
images = []
labels = []

for folder in os.listdir(rootdir):
    classes = os.path.join(rootdir,folder)
    label = folder
    for file in os.listdir(classes):
        img = os.path.join(classes, file)
        image = cv2.imread(img)
        image = preprocess1(image)
        images.append(image)
        labels.append(label)

encoder = LabelEncoder()
labels = encoder.fit_transform(labels)


In [4]:
X = np.array(images)
y = np.array(labels) 
from sklearn.model_selection import train_test_split
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [5]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


# Step 1: Feature Extraction
def extract_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    # Check if descriptors is not None and has at least one row
    if descriptors is not None and descriptors.shape[0] > 0:
        return descriptors
    else:
        return np.zeros((1, 128), dtype=np.float32) # Return empty array of shape (0, 128)

# Step 2: Feature Clustering
def learn_visual_words(train_images, num_words):
    # Concatenate features from all images
    descriptors = np.concatenate([extract_features(img) for img in train_images])
    # Cluster features into visual words
    kmeans = KMeans(n_clusters=num_words, random_state=0).fit(descriptors.astype(np.float32))
    return kmeans

# Step 3: Feature Quantization
def quantize_features(features, kmeans):
    visual_words = kmeans.predict(features)
    return visual_words.astype(np.int32)

# Step 4: Histogram Creation
def create_histogram(visual_words, num_words):
    histogram = np.zeros(num_words, dtype=np.float32)
    visual_words = visual_words.astype(np.int32)  # convert visual_words to int32
    for word in visual_words:
        histogram[word] += 1
    return histogram

# Step 5: Classifier Training and Testing
def train_classifier(train_histograms, train_labels):
    svm = LinearSVC()
    svm.fit(train_histograms.astype(np.float32), train_labels.astype(np.float32))
    return svm

def test_classifier(svm, test_histograms, test_labels):
    pred_labels = svm.predict(test_histograms.astype(np.float32))
    print(classification_report(test_labels.astype(np.float32), pred_labels.astype(np.float32)))
    accuracy = accuracy_score(test_labels.astype(np.float32), pred_labels.astype(np.float32))
    return accuracy

# Load train and test images
train_images = X_train
train_labels = y_train

test_images = X_test
test_labels = y_test

# Learn visual words
num_words = 200
kmeans = learn_visual_words(train_images, num_words)


In [6]:
# Compute histograms for train and test images
train_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in train_images])
test_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in test_images])

# Train and test SVM classifier
svm = train_classifier(train_histograms, train_labels)
accuracy = test_classifier(svm, test_histograms, test_labels)

print("Accuracy: %.2f%%" % (accuracy * 100))

              precision    recall  f1-score   support

         0.0       0.61      0.70      0.65       431
         1.0       0.64      0.74      0.69       439
         2.0       0.56      0.44      0.49       436
         3.0       0.63      0.66      0.64       462
         4.0       0.66      0.53      0.59       459
         5.0       0.62      0.67      0.64       443
         6.0       0.73      0.68      0.70       476
         7.0       0.63      0.64      0.64       448
         8.0       0.67      0.68      0.67       429
         9.0       0.75      0.81      0.78       437
        10.0       0.60      0.58      0.59       490

    accuracy                           0.65      4950
   macro avg       0.64      0.65      0.64      4950
weighted avg       0.64      0.65      0.64      4950

Accuracy: 64.55%


/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [7]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


# Step 1: Feature Extraction
def extract_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    # Check if descriptors is not None and has at least one row
    if descriptors is not None and descriptors.shape[0] > 0:
        return descriptors
    else:
        return np.zeros((1, 128), dtype=np.float32) # Return empty array of shape (0, 128)

# Step 2: Feature Clustering
def learn_visual_words(train_images, num_words):
    # Concatenate features from all images
    descriptors = np.concatenate([extract_features(img) for img in train_images])
    # Cluster features into visual words
    kmeans = KMeans(n_clusters=num_words, random_state=0).fit(descriptors.astype(np.float32))
    return kmeans

# Step 3: Feature Quantization
def quantize_features(features, kmeans):
    visual_words = kmeans.predict(features)
    return visual_words.astype(np.int32)

# Step 4: Histogram Creation
def create_histogram(visual_words, num_words):
    histogram = np.zeros(num_words, dtype=np.float32)
    visual_words = visual_words.astype(np.int32)  # convert visual_words to int32
    for word in visual_words:
        histogram[word] += 1
    return histogram

# Step 5: Classifier Training and Testing
def train_classifier(train_histograms, train_labels):
    svm = LinearSVC()
    svm.fit(train_histograms.astype(np.float32), train_labels.astype(np.float32))
    return svm

def test_classifier(svm, test_histograms, test_labels):
    pred_labels = svm.predict(test_histograms.astype(np.float32))
    print(classification_report(test_labels.astype(np.float32), pred_labels.astype(np.float32)))
    accuracy = accuracy_score(test_labels.astype(np.float32), pred_labels.astype(np.float32))
    return accuracy

# Load train and test images
train_images = X_train
train_labels = y_train

test_images = X_test
test_labels = y_test

# Learn visual words
num_words = 500
kmeans = learn_visual_words(train_images, num_words)


In [8]:
# Compute histograms for train and test images
train_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in train_images])
test_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in test_images])

# Train and test SVM classifier
svm = train_classifier(train_histograms, train_labels)
accuracy = test_classifier(svm, test_histograms, test_labels)

print("Accuracy: %.2f%%" % (accuracy * 100))

              precision    recall  f1-score   support

         0.0       0.70      0.80      0.75       431
         1.0       0.73      0.81      0.77       439
         2.0       0.69      0.58      0.63       436
         3.0       0.71      0.75      0.73       462
         4.0       0.72      0.69      0.70       459
         5.0       0.76      0.78      0.77       443
         6.0       0.78      0.76      0.77       476
         7.0       0.72      0.75      0.74       448
         8.0       0.81      0.77      0.79       429
         9.0       0.82      0.83      0.83       437
        10.0       0.64      0.57      0.60       490

    accuracy                           0.73      4950
   macro avg       0.73      0.74      0.73      4950
weighted avg       0.73      0.73      0.73      4950

Accuracy: 73.33%


/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [5]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


# Step 1: Feature Extraction
def extract_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    # Check if descriptors is not None and has at least one row
    if descriptors is not None and descriptors.shape[0] > 0:
        return descriptors
    else:
        return np.zeros((1, 128), dtype=np.float32) # Return empty array of shape (0, 128)

# Step 2: Feature Clustering
def learn_visual_words(train_images, num_words):
    # Concatenate features from all images
    descriptors = np.concatenate([extract_features(img) for img in train_images])
    # Cluster features into visual words
    kmeans = KMeans(n_clusters=num_words, random_state=0).fit(descriptors.astype(np.float32))
    return kmeans

# Step 3: Feature Quantization
def quantize_features(features, kmeans):
    visual_words = kmeans.predict(features)
    return visual_words.astype(np.int32)

# Step 4: Histogram Creation
def create_histogram(visual_words, num_words):
    histogram = np.zeros(num_words, dtype=np.float32)
    visual_words = visual_words.astype(np.int32)  # convert visual_words to int32
    for word in visual_words:
        histogram[word] += 1
    return histogram

# Step 5: Classifier Training and Testing
def train_classifier(train_histograms, train_labels):
    svm = LinearSVC()
    svm.fit(train_histograms.astype(np.float32), train_labels.astype(np.float32))
    return svm

def test_classifier(svm, test_histograms, test_labels):
    pred_labels = svm.predict(test_histograms.astype(np.float32))
    print(classification_report(test_labels.astype(np.float32), pred_labels.astype(np.float32)))
    accuracy = accuracy_score(test_labels.astype(np.float32), pred_labels.astype(np.float32))
    return accuracy

# Load train and test images
train_images = X_train
train_labels = y_train

test_images = X_test
test_labels = y_test

# Learn visual words
num_words = 1000
kmeans = learn_visual_words(train_images, num_words)


In [6]:
# Compute histograms for train and test images
train_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in train_images])
test_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in test_images])

# Train and test SVM classifier
svm = train_classifier(train_histograms, train_labels)
accuracy = test_classifier(svm, test_histograms, test_labels)

print("Accuracy: %.2f%%" % (accuracy * 100))

              precision    recall  f1-score   support

         0.0       0.77      0.79      0.78       431
         1.0       0.78      0.82      0.80       439
         2.0       0.69      0.67      0.68       436
         3.0       0.78      0.80      0.79       462
         4.0       0.73      0.73      0.73       459
         5.0       0.79      0.80      0.80       443
         6.0       0.79      0.79      0.79       476
         7.0       0.78      0.80      0.79       448
         8.0       0.86      0.79      0.82       429
         9.0       0.84      0.85      0.84       437
        10.0       0.66      0.63      0.64       490

    accuracy                           0.77      4950
   macro avg       0.77      0.77      0.77      4950
weighted avg       0.77      0.77      0.77      4950

Accuracy: 76.95%


/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [10]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


# Step 1: Feature Extraction
def extract_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    # Check if descriptors is not None and has at least one row
    if descriptors is not None and descriptors.shape[0] > 0:
        return descriptors
    else:
        return np.zeros((1, 128), dtype=np.float32) # Return empty array of shape (0, 128)

# Step 2: Feature Clustering
def learn_visual_words(train_images, num_words):
    # Concatenate features from all images
    descriptors = np.concatenate([extract_features(img) for img in train_images])
    # Cluster features into visual words
    kmeans = KMeans(n_clusters=num_words, random_state=0).fit(descriptors.astype(np.float32))
    return kmeans

# Step 3: Feature Quantization
def quantize_features(features, kmeans):
    visual_words = kmeans.predict(features)
    return visual_words.astype(np.int32)

# Step 4: Histogram Creation
def create_histogram(visual_words, num_words):
    histogram = np.zeros(num_words, dtype=np.float32)
    visual_words = visual_words.astype(np.int32)  # convert visual_words to int32
    for word in visual_words:
        histogram[word] += 1
    return histogram

# Step 5: Classifier Training and Testing
def train_classifier(train_histograms, train_labels):
    svm = LinearSVC()
    svm.fit(train_histograms.astype(np.float32), train_labels.astype(np.float32))
    return svm

def test_classifier(svm, test_histograms, test_labels):
    pred_labels = svm.predict(test_histograms.astype(np.float32))
    print(classification_report(test_labels.astype(np.float32), pred_labels.astype(np.float32)))
    accuracy = accuracy_score(test_labels.astype(np.float32), pred_labels.astype(np.float32))
    return accuracy

# Load train and test images
train_images = X_train
train_labels = y_train

test_images = X_test
test_labels = y_test

# Learn visual words
num_words = 700
kmeans = learn_visual_words(train_images, num_words)


In [11]:
# Compute histograms for train and test images
train_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in train_images])
test_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in test_images])

# Train and test SVM classifier
svm = train_classifier(train_histograms, train_labels)
accuracy = test_classifier(svm, test_histograms, test_labels)

print("Accuracy: %.2f%%" % (accuracy * 100))

              precision    recall  f1-score   support

         0.0       0.76      0.78      0.77       431
         1.0       0.79      0.81      0.80       439
         2.0       0.67      0.66      0.67       436
         3.0       0.73      0.74      0.73       462
         4.0       0.69      0.67      0.68       459
         5.0       0.75      0.77      0.76       443
         6.0       0.77      0.79      0.78       476
         7.0       0.78      0.78      0.78       448
         8.0       0.78      0.79      0.78       429
         9.0       0.81      0.82      0.82       437
        10.0       0.65      0.60      0.63       490

    accuracy                           0.74      4950
   macro avg       0.74      0.75      0.74      4950
weighted avg       0.74      0.74      0.74      4950

Accuracy: 74.40%


/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [12]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


# Step 1: Feature Extraction
def extract_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    # Check if descriptors is not None and has at least one row
    if descriptors is not None and descriptors.shape[0] > 0:
        return descriptors
    else:
        return np.zeros((1, 128), dtype=np.float32) # Return empty array of shape (0, 128)

# Step 2: Feature Clustering
def learn_visual_words(train_images, num_words):
    # Concatenate features from all images
    descriptors = np.concatenate([extract_features(img) for img in train_images])
    # Cluster features into visual words
    kmeans = KMeans(n_clusters=num_words, random_state=0).fit(descriptors.astype(np.float32))
    return kmeans

# Step 3: Feature Quantization
def quantize_features(features, kmeans):
    visual_words = kmeans.predict(features)
    return visual_words.astype(np.int32)

# Step 4: Histogram Creation
def create_histogram(visual_words, num_words):
    histogram = np.zeros(num_words, dtype=np.float32)
    visual_words = visual_words.astype(np.int32)  # convert visual_words to int32
    for word in visual_words:
        histogram[word] += 1
    return histogram

# Step 5: Classifier Training and Testing
def train_classifier(train_histograms, train_labels):
    svm = LinearSVC()
    svm.fit(train_histograms.astype(np.float32), train_labels.astype(np.float32))
    return svm

def test_classifier(svm, test_histograms, test_labels):
    pred_labels = svm.predict(test_histograms.astype(np.float32))
    print(classification_report(test_labels.astype(np.float32), pred_labels.astype(np.float32)))
    accuracy = accuracy_score(test_labels.astype(np.float32), pred_labels.astype(np.float32))
    return accuracy

# Load train and test images
train_images = X_train
train_labels = y_train

test_images = X_test
test_labels = y_test

# Learn visual words
num_words = 1200
kmeans = learn_visual_words(train_images, num_words)


In [13]:
# Compute histograms for train and test images
train_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in train_images])
test_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in test_images])

# Train and test SVM classifier
svm = train_classifier(train_histograms, train_labels)
accuracy = test_classifier(svm, test_histograms, test_labels)

print("Accuracy: %.2f%%" % (accuracy * 100))

              precision    recall  f1-score   support

         0.0       0.78      0.79      0.78       431
         1.0       0.78      0.83      0.81       439
         2.0       0.71      0.69      0.70       436
         3.0       0.82      0.81      0.82       462
         4.0       0.77      0.74      0.75       459
         5.0       0.81      0.83      0.82       443
         6.0       0.80      0.82      0.81       476
         7.0       0.83      0.82      0.83       448
         8.0       0.82      0.84      0.83       429
         9.0       0.84      0.86      0.85       437
        10.0       0.68      0.63      0.65       490

    accuracy                           0.79      4950
   macro avg       0.79      0.79      0.79      4950
weighted avg       0.79      0.79      0.79      4950

Accuracy: 78.65%


/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [14]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


# Step 1: Feature Extraction
def extract_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    # Check if descriptors is not None and has at least one row
    if descriptors is not None and descriptors.shape[0] > 0:
        return descriptors
    else:
        return np.zeros((1, 128), dtype=np.float32) # Return empty array of shape (0, 128)

# Step 2: Feature Clustering
def learn_visual_words(train_images, num_words):
    # Concatenate features from all images
    descriptors = np.concatenate([extract_features(img) for img in train_images])
    # Cluster features into visual words
    kmeans = KMeans(n_clusters=num_words, random_state=0).fit(descriptors.astype(np.float32))
    return kmeans

# Step 3: Feature Quantization
def quantize_features(features, kmeans):
    visual_words = kmeans.predict(features)
    return visual_words.astype(np.int32)

# Step 4: Histogram Creation
def create_histogram(visual_words, num_words):
    histogram = np.zeros(num_words, dtype=np.float32)
    visual_words = visual_words.astype(np.int32)  # convert visual_words to int32
    for word in visual_words:
        histogram[word] += 1
    return histogram

# Step 5: Classifier Training and Testing
def train_classifier(train_histograms, train_labels):
    svm = LinearSVC()
    svm.fit(train_histograms.astype(np.float32), train_labels.astype(np.float32))
    return svm

def test_classifier(svm, test_histograms, test_labels):
    pred_labels = svm.predict(test_histograms.astype(np.float32))
    print(classification_report(test_labels.astype(np.float32), pred_labels.astype(np.float32)))
    accuracy = accuracy_score(test_labels.astype(np.float32), pred_labels.astype(np.float32))
    return accuracy

# Load train and test images
train_images = X_train
train_labels = y_train

test_images = X_test
test_labels = y_test

# Learn visual words
num_words = 1600
kmeans = learn_visual_words(train_images, num_words)


In [15]:
# Compute histograms for train and test images
train_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in train_images])
test_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in test_images])

# Train and test SVM classifier
svm = train_classifier(train_histograms, train_labels)
accuracy = test_classifier(svm, test_histograms, test_labels)

print("Accuracy: %.2f%%" % (accuracy * 100))

              precision    recall  f1-score   support

         0.0       0.82      0.84      0.83       431
         1.0       0.82      0.85      0.84       439
         2.0       0.74      0.73      0.73       436
         3.0       0.81      0.83      0.82       462
         4.0       0.76      0.77      0.77       459
         5.0       0.84      0.86      0.85       443
         6.0       0.85      0.83      0.84       476
         7.0       0.84      0.84      0.84       448
         8.0       0.86      0.84      0.85       429
         9.0       0.89      0.87      0.88       437
        10.0       0.71      0.68      0.69       490

    accuracy                           0.81      4950
   macro avg       0.81      0.81      0.81      4950
weighted avg       0.81      0.81      0.81      4950

Accuracy: 81.23%


/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
